In [1]:
base_path <- "/home/jovyan/storage/data"

# get the cell meta-data
path <- paste0(base_path, "/seurat_cell_meta_data.csv")
cell_meta_data <- read.csv(file = path, row.names = 1)

# load the cell-to-cell mapping matrix
path <- paste0(base_path, "/cell-to-cell_mapping.csv")
gw <- read.csv2(path, sep = ",", dec = ".", stringsAsFactors = FALSE, header = T, row.names = 1)
rownames(gw) <- as.character(sapply(X = rownames(gw), FUN = function(x){strsplit(x = x, ".", fixed=T)[[1]][1]}))

# load the
path <- paste0(base_path, "/confocal_states0-FilterWUSCLVtop100.csv")
ref <- read.csv2(path, row.names = 1, sep = ",")

In [2]:
# normalize locations (confocal cells)
gw1 <- gw/colSums(gw)  # make columns (physical cells) sum to 1

In [3]:
# create named vector mapping cells to clusters for scRNA-seq DGE
meta <- as.character(cell_meta_data$cluster_annotation)
names(meta) <- substr(cell_meta_data$cell_id, start = 1, stop = 16)

In [4]:
# obtain cluster numbers for cells in gw matrix
MATCH <- match(rownames(gw), names(meta))
cluster_numbers <- as.character(meta[MATCH])

In [5]:
# summed probabilities over all cells in 3D meristem for all clusters
prob_cluster_df <- NULL
for(cluster in unique(cluster_numbers)){
  gw_tmp <- gw1[cluster_numbers %in% cluster,]
  prob_cluster_df <- cbind(prob_cluster_df, colSums(gw_tmp))
}
colnames(prob_cluster_df) <- unique(cluster_numbers)

In [6]:
# add coordinates of cells
prob_cluster_df <- data.frame(ref[,1:3], prob_cluster_df)

In [7]:
prob_cluster_df[1:4,1:5]

,x,y,z,cluster_1___.pro..cambium,cluster_3___dividing.cells
,<chr>,<chr>,<chr>,<dbl>,<dbl>
50,90.261,67.545,5.04944,0.1344298,0.1050708
51,58.4303,97.106,5.19082,0.1312728,0.1064349
52,47.9124,72.6678,5.50022,0.1427766,0.1053820
53,58.578,88.0824,5.7229,0.1366340,0.1070309


In [8]:
# save the generated cluster projections
path <- paste0(base_path, "/cluster_assignment_3d.csv")
write.csv(x = prob_cluster_df, file = path)